# Data Pre-processing

In [2]:
# Import libraries

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats

C:\Users\deric\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
data_dir = 'data'

In [140]:
df = pd.read_csv(os.path.join(data_dir, 'vehicles_concat.csv'))

df

,price,year,model,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color
0,24895.0,2019.0,jeep-cherokee,unspecified,4 cylinders,gas,6973.0,automatic,4wd,unspecified,SUV,white
1,12999.0,2016.0,nissan-altima,unspecified,unspecified,gas,63810.0,unspecified,fwd,unspecified,sedan,unspecified
2,12799.0,2017.0,hyundai-elantra,unspecified,unspecified,gas,80210.0,automatic,unspecified,unspecified,sedan,silver
3,17850.0,2016.0,ford-f150,unspecified,6 cylinders,gas,76393.0,automatic,rwd,unspecified,unspecified,white
4,24150.0,2016.0,dodge-challenger,unspecified,unspecified,gas,74540.0,automatic,rwd,unspecified,coupe,white
...,...,...,...,...,...,...,...,...,...,...,...,...
106877,33064.0,2019.0,ram-1500,unspecified,unspecified,gas,42920.0,automatic,4wd,unspecified,unspecified,black
106878,27987.0,2018.0,toyota-tacoma,unspecified,unspecified,unspecified,44444.0,automatic,rwd,unspecified,unspecified,unspecified
106879,32663.0,2018.0,chevrolet-traverse,unspecified,unspecified,gas,18498.0,automatic,4wd,unspecified,SUV,silver
106880,22986.0,2019.0,chevrolet-equinox,unspecified,unspecified,gas,21598.0,automatic,4wd,unspecified,SUV,white


In [141]:
def one_hot_encode(df, feature):
    dummies = pd.get_dummies(df[feature], prefix=feature)
    
    df = pd.concat([df, dummies], axis=1)
    
    return df

In [142]:
encoded_df = df.copy()

features = ['model', 'transmission', 'paint_color', 'fuel', 'type', 'drive', 'size', 'condition']

for feature in features:
    encoded_df = one_hot_encode(encoded_df, feature)
    
encoded_df = encoded_df.drop(columns=features)

encoded_df

,price,year,cylinders,odometer,model_acura-ilx,model_acura-mdx,model_acura-rdx,model_acura-tlx,model_audi-a3,model_audi-a4,...,size_mid-size,size_sub-compact,size_unspecified,condition_excellent,condition_fair,condition_good,condition_like new,condition_new,condition_salvage,condition_unspecified
0,24895.0,2019.0,4 cylinders,6973.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,12999.0,2016.0,unspecified,63810.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,12799.0,2017.0,unspecified,80210.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,17850.0,2016.0,6 cylinders,76393.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,24150.0,2016.0,unspecified,74540.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106877,33064.0,2019.0,unspecified,42920.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
106878,27987.0,2018.0,unspecified,44444.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
106879,32663.0,2018.0,unspecified,18498.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
106880,22986.0,2019.0,unspecified,21598.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [153]:
encoded_df.to_csv(os.path.join(data_dir, 'vehicles_encoded.csv'), index=False)

In [156]:
encoded_df

,price,odometer,model_acura-ilx,model_acura-mdx,model_acura-rdx,model_acura-tlx,model_audi-a3,model_audi-a4,model_audi-a5,model_audi-a6,...,size_mid-size,size_sub-compact,size_unspecified,condition_excellent,condition_fair,condition_good,condition_like new,condition_new,condition_salvage,condition_unspecified
0,24895.0,6973.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,12999.0,63810.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,12799.0,80210.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,17850.0,76393.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,24150.0,74540.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106877,33064.0,42920.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
106878,27987.0,44444.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
106879,32663.0,18498.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
106880,22986.0,21598.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [154]:
from sklearn.model_selection import train_test_split

y = pd.DataFrame(encoded_df['price'])
X = encoded_df.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=1/3)

print(X_train.shape)
print(X_validation.shape)
print(X_test.shape)

(53440, 300)
(26721, 300)
(26721, 300)


In [155]:
X_test.to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)
y_test.to_csv(os.path.join(data_dir, 'test_y.csv'), header=False, index=False)

pd.concat([y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)
pd.concat([y_validation, X_validation], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)